# Assignment 3 Ruisi_Gu -- Sub-Notebook Two

## Part Three -- Fit in Models and Comparison

This is the beginning of the second notebook. In this part, I am using H2O to do the machine learning work, which is *H2OAutoML*, also I will generate and compare these models to get the best result. (What need to illustrate is that work which is done in this notebook is based on the dataset generated from the first notebook.)

#### Initialize H2O

In [1]:
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil
import numpy as np

In [2]:
pct_memory=8
virtual_memory=psutil.virtual_memory()
min_mem_size=int(round(int(pct_memory*virtual_memory.available)/1073741824,0))
print(min_mem_size)

12


In [3]:
h2o.init(strict_version_check=False,max_mem_size=min_mem_size)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_191"; Java(TM) SE Runtime Environment (build 1.8.0_191-b12); Java HotSpot(TM) 64-Bit Server VM (build 25.191-b12, mixed mode)
  Starting server from /Users/guruisi/anaconda3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/25/079nl8ps1d17m_b9mm9cwnhm0000gn/T/tmp0pgzh4ov
  JVM stdout: /var/folders/25/079nl8ps1d17m_b9mm9cwnhm0000gn/T/tmp0pgzh4ov/h2o_guruisi_started_from_python.out
  JVM stderr: /var/folders/25/079nl8ps1d17m_b9mm9cwnhm0000gn/T/tmp0pgzh4ov/h2o_guruisi_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,03 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.5
H2O cluster version age:,29 days
H2O cluster name:,H2O_from_python_guruisi_92tcg0
H2O cluster total nodes:,1
H2O cluster free memory:,10.67 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [38]:
# train_data_path = "assignment3/data/train.tsv"
# test_data_path = "assignment3/data/test.tsv"
train_data_path = "assignment3/train_submission.csv"
test_data_path = "assignment3/submission06.csv"

# Load data into H2O
df_train = h2o.import_file(train_data_path)
df_test = h2o.import_file(test_data_path)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
df_train.head()

PhraseId,SentenceId,Phrase,Sentiment,SentimentM1
1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1,3
2,1,A series of escapades demonstrating the adage that what is good for the goose,2,2
3,1,A series,2,2
4,1,A,2,2
5,1,series,2,2
6,1,of escapades demonstrating the adage that what is good for the goose,2,2
7,1,of,2,2
8,1,escapades demonstrating the adage that what is good for the goose,2,2
9,1,escapades,2,2
10,1,demonstrating the adage that what is good for the goose,2,2


In [39]:
df_test.head()

PhraseId,SentimentM1
156061,3
156062,3
156063,2
156064,3
156065,3
156066,3
156067,3
156068,3
156069,3
156070,2


In our case, although Sentiment labels range from 0 to 4, Sentiment is not represent values 0,1,2,3,4. Thus, we need to convert the numbers into classifiers in case H2O consider the data as linear regression.

In [7]:
df_train['Sentiment'] = df_train['Sentiment'].asfactor()

In [8]:
df_train.head()

PhraseId,SentenceId,Phrase,Sentiment,SentimentM1
1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1,3
2,1,A series of escapades demonstrating the adage that what is good for the goose,2,2
3,1,A series,2,2
4,1,A,2,2
5,1,series,2,2
6,1,of escapades demonstrating the adage that what is good for the goose,2,2
7,1,of,2,2
8,1,escapades demonstrating the adage that what is good for the goose,2,2
9,1,escapades,2,2
10,1,demonstrating the adage that what is good for the goose,2,2


#### Processing data by using H2O AutoML. 
Set runtime 15000 seconds in order to get a better model. Then choose the best model from leaderboard as below.

In [9]:
def get_independent_variables(df, targ):
    C = [name for name in df.columns if name != targ]
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in C:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    x=ints+enums+reals
    return x

In [10]:
target='Sentiment'
y = target

In [11]:
X=get_independent_variables(df_train, target) 
print(X)

['PhraseId', 'SentenceId', 'SentimentM1', 'Phrase']


In [12]:
run_time=1500

In [13]:
review_train, review_test = df_train.split_frame([0.1])
review_train.shape

(15610, 5)

In [14]:
# Set up AutoML
aml = H2OAutoML(max_runtime_secs=run_time)

In [15]:
aml.train(x=X,y=y,training_frame=df_train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [16]:
print(aml.leaderboard)

model_id,mean_per_class_error,logloss,rmse,mse
XGBoost_1_AutoML_20190403_185203,0.518557,1.05243,0.614541,0.377661
GLM_grid_1_AutoML_20190403_185203_model_1,0.519231,1.03804,0.59256,0.351127
GBM_1_AutoML_20190403_185203,0.521965,1.00825,0.588014,0.34576
XGBoost_3_AutoML_20190403_185203,0.521969,1.11105,0.645938,0.417236
XGBoost_2_AutoML_20190403_185203,0.522008,1.11353,0.645934,0.41723
GBM_2_AutoML_20190403_185203,0.52578,1.03058,0.602379,0.362861
GBM_3_AutoML_20190403_185203,0.526781,1.01908,0.597172,0.356614
DRF_1_AutoML_20190403_185203,0.527867,0.94295,0.557172,0.31044
StackedEnsemble_AllModels_AutoML_20190403_185203,0.528813,0.908494,0.552658,0.305431
StackedEnsemble_BestOfFamily_AutoML_20190403_185203,0.530084,0.908995,0.552754,0.305537


Based on *mean_per_class_error* and *logloss*, I've selected two models that perform well. One is XGBoost model, which run the first place as we can see from the leaderboard. The other one is GBM (Gradient Boosting Model) which has the least logloss.<br><br>
I will name the **XGBoost** Model as **mod_best**, at the same time, **GBM** Model as **model**

In [17]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[0])

In [18]:
mod_best

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_1_AutoML_20190403_185203


ModelMetricsMultinomial: xgboost
** Reported on train data. **

MSE: 0.3782131116029184
RMSE: 0.6149903345605672
LogLoss: 1.0533676846424738
Mean Per-Class Error: 0.5222057126000716
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



0,1,2,3,4,Error,Rate
2183.0,2773.0,1914.0,176.0,26.0,0.6913179,"4,889 / 7,072"
1218.0,10746.0,13981.0,1234.0,94.0,0.6059839,"16,527 / 27,273"
381.0,4641.0,68415.0,5765.0,380.0,0.1403207,"11,167 / 79,582"
69.0,1135.0,14567.0,15289.0,1867.0,0.5356698,"17,638 / 32,927"
7.0,159.0,1694.0,4011.0,3335.0,0.6377363,"5,871 / 9,206"
3858.0,19454.0,100571.0,26475.0,5702.0,0.3594259,"56,092 / 156,060"


Top-5 Hit Ratios: 


k,hit_ratio
1,0.6405742
2,0.8264257
3,0.9588684
4,0.9882417
5,1.0



ModelMetricsMultinomial: xgboost
** Reported on cross-validation data. **

MSE: 0.37766073429299074
RMSE: 0.6145410761641493
LogLoss: 1.0524307259660532
Mean Per-Class Error: 0.5185567303270829
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



...

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
accuracy,0.6404908,0.0015913,0.6403306,0.6427656,0.6362617,0.6415802,0.6415161
err,0.3595092,0.0015913,0.3596694,0.3572344,0.3637383,0.3584198,0.3584839
err_count,11221.0,49.668903,11226.0,11150.0,11353.0,11187.0,11189.0
logloss,1.0524307,0.0036143,1.0484085,1.0567706,1.0600762,1.0467975,1.0501008
max_per_class_error,0.6658181,0.0026072,0.6666667,0.6587473,0.6687719,0.6661891,0.6687158
mean_per_class_accuracy,0.4814389,0.0024836,0.4831356,0.4856095,0.4761088,0.4837058,0.4786349
mean_per_class_error,0.5185611,0.0024836,0.5168644,0.5143905,0.5238911,0.5162942,0.5213652
mse,0.3776607,0.0023251,0.3752404,0.3810071,0.3822803,0.3745079,0.3752679
r2,0.5272721,0.0037810,0.532164,0.5237573,0.5187320,0.5288429,0.5328644
rmse,0.6145353,0.0018900,0.6125687,0.6172578,0.6182882,0.6119705,0.6125911


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error
,2019-04-03 19:07:35,1 min 58.716 sec,0.0,0.8000000,1.6094379,0.7890106
,2019-04-03 19:07:36,1 min 59.939 sec,5.0,0.7679990,1.4678020,0.4058503
,2019-04-03 19:07:37,2 min 1.001 sec,10.0,0.7386759,1.3609946,0.3599577
,2019-04-03 19:07:38,2 min 2.007 sec,15.0,0.7136354,1.2824532,0.3594130
,2019-04-03 19:07:39,2 min 3.076 sec,20.0,0.6905408,1.2184446,0.3594066
,2019-04-03 19:07:41,2 min 4.204 sec,25.0,0.6736439,1.1760474,0.3594259
,2019-04-03 19:07:42,2 min 5.407 sec,30.0,0.6602159,1.1450163,0.3592144
,2019-04-03 19:07:43,2 min 6.636 sec,35.0,0.6481569,1.1184831,0.3594323
,2019-04-03 19:07:44,2 min 7.938 sec,40.0,0.6379404,1.0973963,0.3595796
,2019-04-03 19:07:46,2 min 9.388 sec,45.0,0.6303097,1.0822860,0.3594130


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
SentimentM1,595629.6875000,1.0,0.9746365
PhraseId,8686.3710938,0.0145835,0.0142136
SentenceId,6814.0507812,0.0114401,0.0111499


In [19]:
mod_best.algo

'xgboost'

In [20]:
model = h2o.get_model(aml.leaderboard[5,'model_id'])
model.algo

'gbm'

In [21]:
model

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_2_AutoML_20190403_185203


ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.3602525771288419
RMSE: 0.600210444035125
LogLoss: 1.023761984731218
Mean Per-Class Error: 0.5212745950931236
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



0,1,2,3,4,Error,Rate
2264.0,2693.0,1914.0,177.0,24.0,0.6798643,"4,808 / 7,072"
1243.0,10727.0,13995.0,1223.0,85.0,0.6066806,"16,546 / 27,273"
377.0,4626.0,68477.0,5762.0,340.0,0.1395416,"11,105 / 79,582"
69.0,1133.0,14577.0,15438.0,1710.0,0.5311447,"17,489 / 32,927"
7.0,157.0,1698.0,4114.0,3230.0,0.6491419,"5,976 / 9,206"
3960.0,19336.0,100661.0,26714.0,5389.0,0.3583494,"55,924 / 156,060"


Top-5 Hit Ratios: 


k,hit_ratio
1,0.6416507
2,0.8330194
3,0.9584583
4,0.9866974
5,1.0



ModelMetricsMultinomial: gbm
** Reported on cross-validation data. **

MSE: 0.362860776738782
RMSE: 0.6023792632044881
LogLoss: 1.0305772465589829
Mean Per-Class Error: 0.5257796504218959
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



...

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
accuracy,0.6398436,0.0016763,0.6393374,0.6417404,0.6354607,0.6418685,0.6408112
err,0.3601564,0.0016763,0.3606626,0.3582596,0.3645393,0.3581315,0.3591888
err_count,11241.2,52.320934,11257.0,11182.0,11378.0,11178.0,11211.0
logloss,1.0305773,0.0015814,1.0305316,1.0299027,1.0341306,1.0271719,1.0311495
max_per_class_error,0.6911059,0.0057290,0.7010014,0.6810655,0.6968421,0.6819484,0.6946721
mean_per_class_accuracy,0.4742235,0.0027388,0.4740294,0.4783736,0.4685284,0.4785430,0.4716432
mean_per_class_error,0.5257765,0.0027388,0.5259706,0.5216265,0.5314716,0.521457,0.5283568
mse,0.3628608,0.0004354,0.3631523,0.3627205,0.363794,0.3619111,0.3627259
r2,0.5458044,0.0015975,0.5472349,0.5466148,0.5420051,0.5446906,0.5484767
rmse,0.602379,0.0003614,0.6026212,0.6022629,0.6031534,0.6015904,0.6022673


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error
,2019-04-03 19:15:11,1 min 1.491 sec,0.0,0.8000000,1.6094379,0.6597591
,2019-04-03 19:15:13,1 min 3.092 sec,5.0,0.7049693,1.2578516,0.3587723
,2019-04-03 19:15:15,1 min 4.923 sec,10.0,0.6470134,1.1143496,0.3588428
,2019-04-03 19:15:17,1 min 6.564 sec,15.0,0.6202089,1.0596427,0.3585031
,2019-04-03 19:15:18,1 min 7.697 sec,20.0,0.6106009,1.0428996,0.3584519
,2019-04-03 19:15:19,1 min 8.675 sec,25.0,0.6028881,1.0289362,0.3582532
,2019-04-03 19:15:19,1 min 9.176 sec,30.0,0.6002104,1.0237620,0.3583494


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
SentimentM1,78767.0625000,1.0,0.9451131
SentenceId,2428.4699707,0.0308310,0.0291388
PhraseId,2145.8820801,0.0272434,0.0257481


In [22]:
# Find the 80th quantile of PhraseId in the dataset
Phrase_split = df_train["PhraseId"].quantile(prob = [0.8])[1]
df_train["Train"] = (df_train["PhraseId"] < Phrase_split).ifelse("Yes", "No")

In [23]:
df_train1 = df_train[df_train["Train"] == "Yes"]
df_test1 = df_train[df_train["Train"] == "No"]

In [24]:
df_train

PhraseId,SentenceId,Phrase,Sentiment,SentimentM1,Train
1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1,3,Yes
2,1,A series of escapades demonstrating the adage that what is good for the goose,2,2,Yes
3,1,A series,2,2,Yes
4,1,A,2,2,Yes
5,1,series,2,2,Yes
6,1,of escapades demonstrating the adage that what is good for the goose,2,2,Yes
7,1,of,2,2,Yes
8,1,escapades demonstrating the adage that what is good for the goose,2,2,Yes
9,1,escapades,2,2,Yes
10,1,demonstrating the adage that what is good for the goose,2,2,Yes


In [25]:
df_train1

PhraseId,SentenceId,Phrase,Sentiment,SentimentM1,Train
1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1,3,Yes
2,1,A series of escapades demonstrating the adage that what is good for the goose,2,2,Yes
3,1,A series,2,2,Yes
4,1,A,2,2,Yes
5,1,series,2,2,Yes
6,1,of escapades demonstrating the adage that what is good for the goose,2,2,Yes
7,1,of,2,2,Yes
8,1,escapades demonstrating the adage that what is good for the goose,2,2,Yes
9,1,escapades,2,2,Yes
10,1,demonstrating the adage that what is good for the goose,2,2,Yes


In [26]:
df_test1

In [40]:
X_train = df_train['Phrase']
y_train = df_train['Sentiment']
X_test = df_test['Phrase']

In [29]:
from h2o.estimators import H2OGradientBoostingEstimator
predictors = ['PhraseId', 'SentenceId', 'SentimentM1', 'Phrase']
response = 'Sentiment'

gbm_embeddings = H2OGradientBoostingEstimator(model_id = "gbm_embeddings.hex")
gbm_embeddings.train(x = predictors, y = response, 
                   training_frame = df_train1, validation_frame = df_test1
                  )

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [30]:
from h2o.estimators import H2OXGBoostEstimator
predictors = ['PhraseId', 'SentenceId', 'SentimentM1', 'Phrase']
response = 'Sentiment'
param = {
      "ntrees" : 100
    , "max_depth" : 10
    , "learn_rate" : 0.02
    , "sample_rate" : 0.7
    , "col_sample_rate_per_tree" : 0.9
    , "min_rows" : 5
    , "seed": 4241
    , "score_tree_interval": 100
}

model_xgboost = H2OXGBoostEstimator(**param)
model_xgboost.train(x = predictors, y = response, 
                   training_frame = df_train1, validation_frame = df_test1)

xgboost Model Build progress: |███████████████████████████████████████████| 100%


In [32]:
cf_xgboost=model_xgboost.confusion_matrix(df_train)
print(cf_xgboost)

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



0,1,2,3,4,Error,Rate
1886.0,3055.0,1506.0,600.0,25.0,0.7333145,"5,186 / 7,072"
985.0,10955.0,11036.0,4209.0,88.0,0.5983207,"16,318 / 27,273"
291.0,4661.0,55543.0,18757.0,330.0,0.3020658,"24,039 / 79,582"
50.0,1138.0,11688.0,18518.0,1533.0,0.4376044,"14,409 / 32,927"
6.0,159.0,1350.0,4524.0,3167.0,0.6559852,"6,039 / 9,206"
3218.0,19968.0,81123.0,46608.0,5143.0,0.4228566,"65,991 / 156,060"


In [34]:
cf=gbm_embeddings.confusion_matrix(df_train)
print(cf)
# print("With Embeddings AUC: " + str(round(gbm_embeddings.auc(valid = True), 3)))

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



0,1,2,3,4,Error,Rate
2346.0,2611.0,1490.0,599.0,26.0,0.6682692,"4,726 / 7,072"
1346.0,10622.0,10992.0,4220.0,93.0,0.6105306,"16,651 / 27,273"
417.0,4587.0,55395.0,18816.0,367.0,0.3039255,"24,187 / 79,582"
76.0,1127.0,11643.0,18258.0,1823.0,0.4455007,"14,669 / 32,927"
7.0,159.0,1342.0,4381.0,3317.0,0.6396915,"5,889 / 9,206"
4192.0,19106.0,80862.0,46274.0,5626.0,0.4236960,"66,122 / 156,060"


In [35]:
# gbm_embeddings.auc(valid = True)
cf=gbm_embeddings.confusion_matrix(df_train)

In [36]:
cf

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



0,1,2,3,4,Error,Rate
2346.0,2611.0,1490.0,599.0,26.0,0.6682692,"4,726 / 7,072"
1346.0,10622.0,10992.0,4220.0,93.0,0.6105306,"16,651 / 27,273"
417.0,4587.0,55395.0,18816.0,367.0,0.3039255,"24,187 / 79,582"
76.0,1127.0,11643.0,18258.0,1823.0,0.4455007,"14,669 / 32,927"
7.0,159.0,1342.0,4381.0,3317.0,0.6396915,"5,889 / 9,206"
4192.0,19106.0,80862.0,46274.0,5626.0,0.4236960,"66,122 / 156,060"


In [ ]:
# cf.auc

In [55]:
# final_result1 = gbm_embeddings.predict(X_test)
predict = model_xgboost.predict(df_test)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [56]:
predict

predict,p0,p1,p2,p3,p4
3,0.0610714,0.0833332,0.101097,0.549312,0.205186
3,0.0610714,0.0833332,0.101097,0.549312,0.205186
2,0.0805012,0.115926,0.398743,0.263018,0.141812
3,0.0610714,0.0833332,0.101097,0.549312,0.205186
3,0.0610714,0.0833332,0.101097,0.549312,0.205186
3,0.0610714,0.0833332,0.101097,0.549312,0.205186
3,0.0610714,0.0833332,0.101097,0.549312,0.205186
3,0.0610714,0.0833332,0.101097,0.549312,0.205186
3,0.0610714,0.0833332,0.101097,0.549312,0.205186
2,0.0805012,0.115926,0.398743,0.263018,0.141812


In [57]:
result_Sentiment = predict['predict']

In [58]:
result_Sentiment

predict
3
3
2
3
3
3
3
3
3
2


In [59]:
type(predict)

h2o.frame.H2OFrame

In [60]:

result_data_path = "assignment3/data/test.tsv"

# Load data into H2O

df_result = h2o.import_file(result_data_path, sep="\t")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [61]:
df_result

PhraseId,SentenceId,Phrase
156061,8545,An intermittently pleasing but mostly routine effort .
156062,8545,An intermittently pleasing but mostly routine effort
156063,8545,An
156064,8545,intermittently pleasing but mostly routine effort
156065,8545,intermittently pleasing but mostly routine
156066,8545,intermittently pleasing but
156067,8545,intermittently pleasing
156068,8545,intermittently
156069,8545,pleasing
156070,8545,but


In [62]:
df_result = df_result.cbind(result_Sentiment)

In [63]:
df_result

PhraseId,SentenceId,Phrase,predict
156061,8545,An intermittently pleasing but mostly routine effort .,3
156062,8545,An intermittently pleasing but mostly routine effort,3
156063,8545,An,2
156064,8545,intermittently pleasing but mostly routine effort,3
156065,8545,intermittently pleasing but mostly routine,3
156066,8545,intermittently pleasing but,3
156067,8545,intermittently pleasing,3
156068,8545,intermittently,3
156069,8545,pleasing,3
156070,8545,but,2


In [64]:
Submission = df_result.drop(['SentenceId','Phrase'])

In [65]:
Submission

PhraseId,predict
156061,3
156062,3
156063,2
156064,3
156065,3
156066,3
156067,3
156068,3
156069,3
156070,2


In [66]:
Submission.set_names(['PhraseId','Sentiment'])

PhraseId,Sentiment
156061,3
156062,3
156063,2
156064,3
156065,3
156066,3
156067,3
156068,3
156069,3
156070,2


In [68]:
h2o.export_file(Submission, 'Submission_plza.csv', force=False)

Export File progress: |███████████████████████████████████████████████████| 100%


# Conclusion

In this assignment, I separated into two parts to finish, the first one is data cleaning and preprocessing, and the second part is to use H2O library to make it better. <br><br>

In the first part, I use STEMMER to clean the data, which changes some miss typing words to correct version. Then, I tokenize the words using CountVectorizer. Therefore, we get the mean cross-validation score from Part One, which is about 0.603. Before entering to part two, I save the predictions I get from part one and save them as a new column, this new dataset is the one we are going to use in part two. <br><br>

Instead of importing the original dataset, we will import the one with new columns generated from Part One and then processing them by using H2O Auto ML. However H2O and sklearn cannot exist on the same notebook, that is the reason we have to create two notebooks and finish the work separately. In H2O part, first we initialize H2O and then set up AutoML. AutoML will help us find out the best model fit in our dataset by compare the rmse, log loss and so on from the leaderboard. In our case, the XGBoost run the first place on our leaderboard so we will run this model and let our dataset fit into it. By using XGBoost model, we predict our dataset result in 0.604.  <br><br>
This is a simple combination of sklearn and H2O, however the overfitting problem is a huge problem I should pay more attention to.

# Contributions

By own: 60%<br>
By external source:40%

# Citations

https://www.kaggle.com/sshadylov/sentiment-analysis-on-rotten-tomatoes-dataset<br>
https://github.com/h2oai/h2o-tutorials/blob/master/training/h2o_3_hands_on/nlp/WordEmbeddings-PreRun.ipynb

# License

Copyright 2019 RuisiGu

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [ ]:
h2o.cluster().shutdown()